In [50]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression

#### 제3유형 로지스틱 회귀분석

In [51]:
import seaborn as sns

df = sns.load_dataset('titanic')

df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [52]:
# 분석 데이터 결정
# 독립변수 : sex, sibsp, fare
# 종속변수 : survived

df = df[['survived', 'sex', 'sibsp', 'fare']]

df.head()

,survived,sex,sibsp,fare
0,0,male,1,7.2500
1,1,female,1,71.2833
2,1,female,0,7.9250
3,1,female,1,53.1000
4,0,male,0,8.0500


In [53]:
# 로지스틱 회귀분석
# y값이 1일 확률을 예측하는 것!
# 회귀식 : P(1일 확률) = 1/(1+exp(-f(x)))

# f(x) = b0 + b1x1 + b2x2 + b3x3
# ln(P/1-P) = b0 + b1x1 + b2x2 + b3x3    (Logit)
# (P = 생존할 확률, x1=sex, x2=sibsp, x3=fare)

In [54]:
# 변수가 문자형인 경우 분석을 진행할 수 없으므로 변환해줘야 한다

# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()

# df['sex'] = le.fit_transform(df['sex'])   # 남성이 1로 책정 반대인 경우

df['sex'] = df['sex'].map({'female':1, 'male':0}) # 여성을 1로 남성을 0으로 설정할 경우

print(df.head())
print(df.info())


   survived  sex  sibsp     fare
0         0    0      1   7.2500
1         1    1      1  71.2833
2         1    1      0   7.9250
3         1    1      1  53.1000
4         0    0      0   8.0500
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   sex       891 non-null    int64  
 2   sibsp     891 non-null    int64  
 3   fare      891 non-null    float64
dtypes: float64(1), int64(3)
memory usage: 28.0 KB
None


In [56]:
from sklearn.linear_model import LogisticRegression

# 주의!!!! LogisticRegression() 객체 안에는 반드시 penalty='none' 옵션을 줘야한다 (default = 'l2')
model = LogisticRegression(penalty='none')  # 버전 이슈 주의! 현재 시험 환경에서는 'none'으로 해줘야한다!
model.fit(df.iloc[:,1:], df.iloc[:,0])

# print(df.iloc[:,1:].head())
# print(df.iloc[:,0])


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


LogisticRegression(penalty='none')

In [61]:
# 로지스틱 회귀분석 관련 지표 출력

print(np.round(model.coef_,4))
print(np.round(model.coef_[0,0],4))
print(np.round(model.coef_[0,1],4))
print(np.round(model.coef_[0,2],4))
print(np.round(model.intercept_[0],4))

[[ 2.5668 -0.4017  0.0138]]
2.5668
-0.4017
0.0138
-1.6964


#### 결과 : Logit = ln(P/1-P) = -1.6964 + 2.5668sex - 0.4017sibp + 0.0138fare

문제.

로지스틱 회귀모형에서 sibsp 변수가 한 단위 증가할 때 생존할 오즈가 몇 배 증가하는지 구하시오
* 반올림하여 소수점 셋째 자리까지 구하시오.

In [63]:
odds_ratio = round(np.exp(model.coef_[0,1]),3)

print(odds_ratio)

0.669


문제.

로지스틱 회귀모형에서 여성일 경우 남성에 비해 오즈가 몇 배 증가하는지 구하시오  (1 값을 어떤 것으로 잡았냐에 따라 달라진다!!!)
* 반올림하여 소수점 셋째자리까지 구하시오.

In [66]:
odds_ratio = round(np.exp(model.coef_[0,0]),3)

print(odds_ratio)

13.024


결과. 여성일 경우 남성에 비해 생존할 오즈가 13.024배 증가한다.

#### by statsmodels

In [72]:
import statsmodels.api as sm   # 선형 : OLS, 로지스틱 : Logit

x = df.iloc[:,1:]
y = df.iloc[:,0]

x = sm.add_constant(x)

model = sm.Logit(y,x).fit()

summary = model.summary()

print(summary)

Optimization terminated successfully.
         Current function value: 0.483846
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      887
Method:                           MLE   Df Model:                            3
Date:                Fri, 01 Dec 2023   Pseudo R-squ.:                  0.2734
Time:                        15:03:41   Log-Likelihood:                -431.11
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 5.094e-70
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6964      0.129    -13.134      0.000      -1.950      -1.443
sex            2.5668      0.